In [22]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import json
import re 
import time

In [2]:
# Carga de datos
match_df = pd.read_pickle('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\match_data_version2.pickle')
winner_df =  pd.read_pickle('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\match_loser_data_version1.pickle')
loser_df = pd.read_pickle('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\match_winner_data_version1.pickle')

In [15]:
# Informacion de los dataframes
match_df.info()
winner_df.info()
loser_df.info()
match_df['participants'].iloc[0][0]['timeline']

<class 'pandas.core.frame.DataFrame'>
Index: 145796 entries, 0 to 0
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   gameCreation           145796 non-null  float64
 1   gameDuration           145796 non-null  float64
 2   gameId                 145796 non-null  int64  
 3   gameMode               145796 non-null  object 
 4   gameType               145796 non-null  object 
 5   gameVersion            145796 non-null  object 
 6   mapId                  145796 non-null  float64
 7   participantIdentities  145796 non-null  object 
 8   participants           145796 non-null  object 
 9   platformId             145796 non-null  object 
 10  queueId                145796 non-null  float64
 11  seasonId               145796 non-null  float64
 12  status.message         0 non-null       object 
 13  status.status_code     0 non-null       float64
 14  teams                  145796 non-null  object

{'participantId': 1,
 'creepsPerMinDeltas': {'10-20': 6.199999999999999, '0-10': 6.1},
 'xpPerMinDeltas': {'10-20': 473.5, '0-10': 386.6},
 'goldPerMinDeltas': {'10-20': 277, '0-10': 315.6},
 'damageTakenPerMinDeltas': {'10-20': 906, '0-10': 473.79999999999995},
 'role': 'DUO_CARRY',
 'lane': 'MIDDLE'}


Seleccionamos las columnas utiles para la prediccion de las partidas
* columnas match_df: [participants, gameVersion, seasonId]
* columnas winner_df: [win, firtsBlood, firstTower, firstInhibitor, firstBaron, firstDragon, firstRiftHerald, towerKills, inhibitorKills, baronKills, dragonKills, riftHeraldKills, bans]
* columnas loser_df: [win, firtsBlood, firstTower, firstInhibitor, firstBaron, firstDragon, firstRiftHerald, towerKills, inhibitorKills, baronKills, dragonKills, riftHeraldKills, bans]

* Dentro de 'participants' utilizaremos los siguientes datos: [championId, spell1Id, spell2Id, timeline], dentro de timeline utilizaremos: [creepsPerMinDeltas, xpPerMinDeltas, goldPerMinDeltas, damageTakenPerMinDeltas, role, lane] donde se utilizara los intervalos de tiempo de los primeros 10 mins.

In [34]:
# Modificamos los nombres de las columnas para evitar confusiones agregando "_winner" y "_loser"
winner_df = winner_df.rename(columns={'win': 'win_winner', 'firstBlood': 'firstBlood_winner', 'firstTower': 'firstTower_winner', 'firstInhibitor': 'firstInhibitor_winner', 'firstBaron': 'firstBaron_winner', 'firstDragon': 'firstDragon_winner', 'firstRiftHerald': 'firstRiftHerald_winner', 'towerKills': 'towerKills_winner', 'inhibitorKills': 'inhibitorKills_winner', 'baronKills': 'baronKills_winner', 'dragonKills': 'dragonKills_winner', 'riftHeraldKills': 'riftHeraldKills_winner', 'bans': 'bans_winner'})
loser_df = loser_df.rename(columns={'win': 'win_loser', 'firstBlood': 'firstBlood_loser', 'firstTower': 'firstTower_loser', 'firstInhibitor': 'firstInhibitor_loser', 'firstBaron': 'firstBaron_loser', 'firstDragon': 'firstDragon_loser', 'firstRiftHerald': 'firstRiftHerald_loser', 'towerKills': 'towerKills_loser', 'inhibitorKills': 'inhibitorKills_loser', 'baronKills': 'baronKills_loser', 'dragonKills': 'dragonKills_loser', 'riftHeraldKills': 'riftHeraldKills_loser', 'bans': 'bans_loser'})


In [37]:
# Seleccionamos las columnas utiles para la prediccion de las partidas
prediction_data = pd.concat([
    match_df[['gameId','participants', 'gameVersion', 'seasonId']].reset_index(drop=True),
    winner_df[['win_winner', 'firstBlood_winner', 'firstTower_winner', 'firstInhibitor_winner', 'firstBaron_winner', 'firstDragon_winner', 'firstRiftHerald_winner', 'towerKills_winner', 'inhibitorKills_winner', 'baronKills_winner', 'dragonKills_winner', 'riftHeraldKills_winner', 'bans_winner']].reset_index(drop=True),
    loser_df[['win_loser', 'firstBlood_loser', 'firstTower_loser', 'firstInhibitor_loser', 'firstBaron_loser', 'firstDragon_loser', 'firstRiftHerald_loser', 'towerKills_loser', 'inhibitorKills_loser', 'baronKills_loser', 'dragonKills_loser', 'riftHeraldKills_loser', 'bans_loser']].reset_index(drop=True)
], axis=1)